# AutoKeras AutoCV  - Experimento

Este componente utiliza [AutoKeras](https://autokeras.com/) AutoCV para a tarefa de classificação.     
O algoritmo faz a busca por arquiteturas e hyperparâmetros que melhor configuram o modelo
para a base de dados fornecida.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

Para esse componente, a base de dados deve estar no seguinte formado:
- Imagens coloridas (3 canais) no formato 256x256 pixels. Caso não estejam nesse formato, o código faz as alterações necesssárias
- Cada classe tem sua pasta com suas respectivas imagens, além dos conjuntos de treino, validação e teste terem suas pastas separadas. Um exemplo da árvore de diretórios pode ser observado abaixo:

```bash
dataset
|________train
|        |_____class_name1
|        |     |____image0.jpg
|        |     |____image1.jpg
|        |     ...
|        |
|        |_____class_name2
|              |____image3.jpg
|              |____image4.jpg
|               ...
|________val
|        |_____class_name1
|        |     |____image5.jpg
|        |     |____image6.jpg
|        |     ...
|        |
|        |_____class_name2
|              |____image7.jpg
|              |____image8.jpg
|               ...
|________test
|        |_____class_name1
|        |     |____image9.jpg
|        |     |____image10.jpg
|        |     ...
|        |
|        |_____class_name2
|              |____image11.jpg
|              |____image12.jpg
|              ...
```

In [1]:
dataset = "/tmp/data/beans_disease.zip" #@param {type:"string"}
num_epochs = 8 # Número de épocas.
trials = 5 # Número de tentativas para o algoritmo de busca por arquiteturas e hyperparametros.
batch_size = 8 # Tamanho do lote de imagens
target_size = (256, 256) # Tamanho das imagens de entrada da rede

'''
[OPCIONAIS]
Aumentações são técnicas que comprovadamente ajudam 
a melhorar o desempenho e generalização dos modelos.
Essas técnicas serão utilizadas apenas nos conjuntos de treino e validação. 
Não utilizadas no conjunto de teste.
Possibilidades de aumentações para utilizar no dataset estão listadas abaixo:

Mais explicações sobre cada aumentação podem ser encontradas no link:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
'''

brightness_range = None # Tupla ou lista de dois float. Range for picking a brightness shift value from.
channel_shift_range = 0.0 # Float. Range for random channel shifts.
cval = 0.0 # Float or Int. Value used for points outside the boundaries when fill_mode = "constant".
data_format = None # Formato dos dados da imagem, pode ser "channels_first" ou "channels_last". Padrão é "channels_last".
dtype = 'float32' # Dtype para usar nos arrays gerados.
featurewise_center = False # Boolean. Set input mean to 0 over the dataset, feature-wise.
featurewise_std_normalization = False # Boolean. Divide entradas pelo desvio padrão (std) do dataset em recursos, feature-wise.
fill_mode = 'nearest' # Opções: {"constant", "nearest", "reflect" or "wrap"}. Padrão é 'nearest'.
horizontal_flip = False # Boolean. Aplica inversão horizontal nas entragas aleatoriamente.
preprocessing_function = None # Funções que serão aplicadas em cada entrada. A função irá rodar após a imagem ser aumentada e redimensionada.
rescale = 1./255 # Fator de re-escala. Padrão é None. Se None ou 0, nenhuma re-escala serã aplicada, caso contrário cada data será multiplicado pelo valor especificado.
rotation_range = 0 # Int. Degree range for random rotations.
samplewise_center = False # Boolean. Define cada média de amostra para 0.
samplewise_std_normalization = False # Boolean. Divide cada entrada por seu desvio padrão.
shear_range = 0.0 # Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees) 
validation_split = 0.0 # Float. Fraction of images reserved for validation (strictly between 0 and 1).
vertical_flip = False # Boolean.  Aplica inversão vertical nas entragas aleatoriamente.
zca_whitening = False # Boolean. Apply ZCA whitening.
zca_epsilon = 1e-06 # epsilon for ZCA whitening. Default is 1e-6.
zoom_range = 0.0 # Float or [lower, upper]. Range for random zoom.
'''
width_shift_range or height_shift_range: Float, 1-D array-like or int. 
- Float: fraction of total width (height) if < 1, or pixels if >= 1. 
- 1-D array-like: random elements from the array. 
- int: integer number of pixels from interval (-shift_range, +shift_range)
'''
height_shift_range = 0.0
width_shift_range = 0.0

In [2]:
!pip install autokeras==1.0.12

In [3]:
import os
import zipfile

root_folder_name = dataset.split("/")[-1].split(".")[0]
root_folder = os.path.join("/tmp/data", root_folder_name)
with zipfile.ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall(root_folder)

### Criação do iterador de dados para a fase de treinamento

In [4]:
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(root_folder, 'train'),
    labels='inferred', label_mode='int',
    batch_size=batch_size)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(root_folder, 'val'),
    labels='inferred', label_mode='int',
    batch_size=batch_size)

2.3.0
Found 1034 files belonging to 3 classes.
Found 133 files belonging to 3 classes.


In [5]:
train_data.class_names

['angular_leaf_spot', 'bean_rust', 'healthy']

### Instanciação do [autokeras](https://autokeras.com/tutorial/image_classification/) para Classificação de Imagens e busca pela melhor arquitetura e hyperparâmetros

In [7]:
import autokeras as ak

model = ak.ImageClassifier(
    num_classes=len(train_data.class_names),
    max_trials=trials,
    metrics="accuracy",
    objective="val_loss",
    overwrite=True)

In [8]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=num_epochs)

Trial 2 Complete [01h 23m 55s]
val_loss: 1.1015214920043945

Best val_loss So Far: 1.0977067947387695
Total elapsed time: 01h 45m 22s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
130/130 [==============================] - 486s 4s/step - loss: 2.5756 - accuracy: 0.3240 - val_loss: 1.0994 - val_accuracy: 0.3158
Epoch 2/2
130/130 [==============================] - 497s 4s/step - loss: 1.0997 - accuracy: 0.3191 - val_loss: 1.0984 - val_accuracy: 0.3459
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


### Exportação do melhor modelo e exposição da sua configuração de camadas

In [9]:
model_exported = model.export_model()
model_exported.summary()
model_path = "/tmp/data/model_weights.h5"
model_exported.save(model_path)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 256, 256, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 256, 256, 3)       7         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 64)     

### Criação dos dataloaders para última fase de treino

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    brightness_range = brightness_range,
    channel_shift_range = channel_shift_range,
    cval = cval,
    data_format = data_format,
    dtype = dtype,
    featurewise_center = featurewise_center,
    featurewise_std_normalization = featurewise_std_normalization,
    fill_mode = fill_mode,
    horizontal_flip = horizontal_flip,
    preprocessing_function = preprocessing_function,
    rescale = rescale,
    rotation_range = rotation_range,
    samplewise_center = False,
    samplewise_std_normalization = samplewise_std_normalization,
    shear_range = shear_range,
    validation_split = validation_split,
    vertical_flip = vertical_flip,
    zca_whitening = zca_whitening,
    zca_epsilon = zca_epsilon,
    zoom_range = zoom_range,
    height_shift_range = height_shift_range,
    width_shift_range = width_shift_range
)

train_generator = image_generator.flow_from_directory(
    os.path.join(root_folder, 'train'),
    batch_size=batch_size,
    target_size=target_size)

validation_generator = image_generator.flow_from_directory(
    os.path.join(root_folder, 'val'),
    batch_size=batch_size,
    target_size=target_size)

test_datagen = ImageDataGenerator(
    rescale=rescale,
    dtype=dtype)

test_generator = test_datagen.flow_from_directory(
    os.path.join(root_folder, 'test'),
    batch_size=batch_size,
    target_size=target_size)

Found 1034 images belonging to 3 classes.
Found 133 images belonging to 3 classes.
Found 128 images belonging to 3 classes.


### Treino do melhor modelo encontrado na busca utilizando Data Augmentation

In [18]:
model = tf.keras.models.load_model(model_path)
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=num_epochs,
    verbose=True)

Epoch 1/2
130/130 [==============================] - 450s 3s/step - loss: 1.0989 - accuracy: 0.3221 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 2/2
130/130 [==============================] - 439s 3s/step - loss: 1.0988 - accuracy: 0.2979 - val_loss: 1.0986 - val_accuracy: 0.3383


### Avaliação do melhor modelo no conjunto de teste

In [35]:
# evaluate the best model
loss, acc = model.evaluate(x=test_generator, verbose=True)
print("Loss: {0} / Acc: {1}".format(loss, acc))

16/16 [==============================] - 8s 529ms/step - loss: 1.0986 - accuracy: 0.3359
Loss, Acc: 1.098594069480896 0.3359375


In [31]:
import numpy as np

predictions = model.predict(x=test_generator)
preds = []
for prediction in predictions:
    indx = np.argmax(prediction)
    preds.append([indx])

### Geração do relatório de classificação e da matrix de confusão 

In [53]:
from sklearn.metrics import classification_report, confusion_matrix

report = classification_report(test_generator.classes, preds, target_names=test_generator.class_indices.keys())
print(report)

In [53]:
c_matrix = confusion_matrix(test_generator.classes, preds)
print("### Confusion matrix:###\n", c_matrix)

                   precision    recall  f1-score   support

angular_leaf_spot       0.00      0.00      0.00        43
        bean_rust       0.34      1.00      0.50        43
          healthy       0.00      0.00      0.00        42

         accuracy                           0.34       128
        macro avg       0.11      0.33      0.17       128
     weighted avg       0.11      0.34      0.17       128

### Confusion matrix:###
 [[ 0 43  0]
 [ 0 43  0]
 [ 0 42  0]]


## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [ ]:
from joblib import dump

artifacts = {
    "model_path": model_path,
}

dump(artifacts, "/tmp/data/model.joblib")